In [1]:
import requests
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

def create_url(authKey, category, **kwargs):
    url = f'https://apihub.kma.go.kr/api/{category}?'
    for key, value in kwargs.items():
        url += f'{key}={value}&'
    url += f'authKey={authKey}'
    return url

authKey='oChwlojFS7aocJaIxcu2ow'
category='typ01/url/kma_sfctm3.php' # ASOS data domain

def create_header(url):
    url = url.replace('help=0','help=1')
    response = requests.get(url)
    res_txt = response.text
    lines = res_txt.strip().split('\n')

    for line_number, line in enumerate(lines):
        if '01234567890123456789' in line:
            endline = line_number-1
            break
    header_rows = [line.split() for line in lines[4:endline]]
    header=[]
    for i in header_rows:
        header.append(i[2])
    return header

data_list = []
start_date = '2009-12-31'
end_date = '2023-12-31'

# 날짜 범위 생성 (1시간 간격)
date_range = pd.date_range(start=start_date, end=end_date, freq='d')

for date in date_range:
    date = date.strftime('%Y%m%d')
    params={
        "tm1": date + '0000',
        "tm2" : date + '2300',
        "stn": 101, # 서울 : 108, 춘천 : 101
        "help": 0,
    }
    url = create_url(authKey, category, **params)

    response = requests.get(url)
    res_txt = response.text # 데이터를 문자열로 변환

    lines = res_txt.strip().split('\n')
    data_rows = [line.split() for line in lines]
    data = data_rows[4:-1]


    header = create_header(url)

    df = pd.DataFrame(data, columns=header)[['TM', 'RN']]
    print('start :', df['TM'].iloc[0], ', end :', df['TM'].iloc[-1])
    data_list.append(df)



Mounted at /content/drive
start : 200912310000 , end : 200912312300
start : 201001010000 , end : 201001012300
start : 201001020000 , end : 201001022300
start : 201001030000 , end : 201001032300
start : 201001040000 , end : 201001042300
start : 201001050000 , end : 201001052300
start : 201001060000 , end : 201001062300
start : 201001070000 , end : 201001072300
start : 201001080000 , end : 201001082300
start : 201001090000 , end : 201001092300
start : 201001100000 , end : 201001102300
start : 201001110000 , end : 201001112300
start : 201001120000 , end : 201001122300
start : 201001130000 , end : 201001132300
start : 201001140000 , end : 201001142300
start : 201001150000 , end : 201001152300
start : 201001160000 , end : 201001162300
start : 201001170000 , end : 201001172300
start : 201001180000 , end : 201001182300
start : 201001190000 , end : 201001192300
start : 201001200000 , end : 201001202300
start : 201001210000 , end : 201001212300
start : 201001220000 , end : 201001222300
start : 

ValueError: 46 columns passed, passed data had 1 columns

ValueError: 46 columns passed, passed data had 1 columns

In [ ]:
data_list = pd.concat(data_list)

In [ ]:
from datetime import datetime
data_list['TM'] = pd.to_datetime(data_list['TM'])  # 문자열을 날짜 및 시간 형식으로 변환
data_list['TM'] = data_list['TM'].dt.strftime('%Y-%m-%d %H:%M:%S')  # 원하는 형식으로 문자열로 변환

In [ ]:
data_list.to_csv('/content/drive/MyDrive/capstone/data/2010~2023_kangwon_precipitation.csv', encoding='utf-8')